# <p style="padding:10px;background-color:#C7BBbC;margin:0;color:white;font-family:newtimeroman;font-size:150%;text-align:center;border-radius: 15px 50px;overflow:hidden;font-weight:500">CNN - Cat & Dog</p>

<p style="text-align:center; ">
<img src="https://media.istockphoto.com/id/1271494334/photo/british-shorthair-and-golden-retriever.jpg?s=612x612&w=0&k=20&c=AoR3uC0zA9R1ku8n3amsv3h9TlKIjg_CnF5K83-YW-s=">
</p>

## Importing necessary libraries

In [ ]:
import warnings
warnings.filterwarnings("ignore") #used to stop unwanted warnings from showing when you execute code

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from tensorflow import keras
from tensorflow.keras.models import Sequential , load_model
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, Activation, BatchNormalization
from keras.utils import load_img

preparing the dataset : we can append 1 and 0 to each category 

In [ ]:
import os

filenames = os.listdir("/kaggle/input/dog-and-cat-dataset/train/train")
categories = []
for filename in filenames:
    category = filename.split('.')[0]
    if category == 'dog':
        categories.append(1)
    else:
        categories.append(0)

df = pd.DataFrame({
    'filename': filenames,
    'category': categories
})

In [ ]:
df.head()

Sample image :

In [ ]:
import random
sample = random.choice(filenames)
image = load_img("/kaggle/input/dog-and-cat-dataset/train/train/"+sample)
plt.imshow(image)

we will convert 1 to dog and 0 to cat

In [ ]:
df["category"] = df["category"].replace({0: 'cat', 1: 'dog'}) 

**we can split the training dataset into train and validation set to avoid overfitting**

In [ ]:
from sklearn.model_selection import train_test_split

train_df, validation_df = train_test_split(df, test_size=0.20, random_state=42)
train_df = train_df.reset_index(drop=True)
validation_df = validation_df.reset_index(drop=True)

In [ ]:
total_train = train_df.shape[0]
total_validate = validation_df.shape[0]

print(total_train, total_validate)

In [ ]:
batch_size=15

## Initializing the model:

In [ ]:
model = Sequential()

model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(128,128,3)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax')) # 2 because we have cat and dog classes

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()

**ImageDataGenerator**

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
# to apply random transformations to each image as it is passed to the model 

In [ ]:
train_datagen = ImageDataGenerator(
    rotation_range=15,
    rescale=1./255,
    shear_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True,
    width_shift_range=0.1,
    height_shift_range=0.1
)

train_generator = train_datagen.flow_from_dataframe(
    train_df, 
    "/kaggle/input/dog-and-cat-dataset/train/train", 
    x_col='filename',
    y_col='category',
    target_size=(128,128),
    class_mode='categorical',
    batch_size=batch_size
)

**validation generator**

In [ ]:
validation_datagen = ImageDataGenerator(rescale=1./255)
validation_generator = validation_datagen.flow_from_dataframe(
    validation_df, 
    "/kaggle/input/dog-and-cat-dataset/train/train/", 
    x_col='filename',
    y_col='category',
    target_size=(128,128),
    class_mode='categorical',
    batch_size=batch_size
)

**Fitting the model**

In [ ]:
history = model.fit_generator(
    train_generator, 
    epochs=50,
    validation_data=validation_generator,
    validation_steps=total_validate//batch_size,
    steps_per_epoch=total_train//batch_size,
)

**Saving the model**

In [ ]:
model.save_weights("model.h5")

**Saving history in case we want to plot the loss or accuracy later on.**

In [ ]:
import pickle
with open('/trainHistoryDict', 'wb') as file_pi:
    pickle.dump(history.history, file_pi)

In [ ]:
# load the history
with open('/trainHistoryDict', "rb") as file_pi:
    history = pickle.load(file_pi)

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 12))
ax1.plot(history['loss'], color='b', label="Training loss")
ax1.plot(history['val_loss'], color='r', label="validation loss")
ax1.set_xticks(np.arange(1, 50, 1))
ax1.set_yticks(np.arange(0, 1, 0.1))

ax2.plot(history['accuracy'], color='b', label="Training accuracy")
ax2.plot(history['val_accuracy'], color='r',label="Validation accuracy")
ax2.set_xticks(np.arange(1, 50, 1))

legend = plt.legend(loc='best', shadow=True)
plt.tight_layout()
plt.show()

**Testing Data**

In [ ]:
test_filenames = os.listdir("/kaggle/input/dog-and-cat-dataset/test1/test1")
test_df = pd.DataFrame({
    'filename': test_filenames
})

In [ ]:
nb_samples = test_df.shape[0]
nb_samples

**Testing Generator**

In [ ]:
test_gen = ImageDataGenerator(rescale=1./255)
test_generator = test_gen.flow_from_dataframe(
    test_df, 
    "/kaggle/input/dog-and-cat-dataset/test1/test1", 
    x_col='filename',
    y_col=None,
    class_mode=None,
    target_size=(128,128),
    batch_size=batch_size,
    shuffle=False
)

**Prediction**

In [ ]:
pred = model.predict_generator(test_generator, steps=np.ceil(nb_samples/batch_size))

category that has the highest probability :

In [ ]:
test_df['category'] = np.argmax(pred, axis=-1)

In [ ]:
label_map = dict((v,k) for k,v in train_generator.class_indices.items())
test_df['category'] = test_df['category'].replace(label_map)

In [ ]:
test_df['category'] = test_df['category'].replace({ 'dog': 1, 'cat': 0 })

### our two classes : 'dog': 1, 'cat': 0 

In [ ]:
sample_test = test_df.head(18)
sample_test.head()
plt.figure(figsize=(12, 24))
for index, row in sample_test.iterrows():
    filename = row['filename']
    category = row['category']
    img = load_img("/kaggle/input/dog-and-cat-dataset/test1/test1/"+filename, target_size=(128,128))
    plt.subplot(6, 3, index+1)
    plt.imshow(img)
    plt.xlabel(filename + '(' + "{}".format(category) + ')' )
plt.tight_layout()
plt.show()